In [34]:
import numpy as np

# Create a NumPy array
array = np.array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])

# Create a shuffled copy of the array along the first dimension
shuffled_array = np.random.permutation(array)

print("Original Array:")
print(array)

print("\nShuffled Array:")
print(shuffled_array)

Original Array:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Shuffled Array:
[[7 8 9]
 [4 5 6]
 [1 2 3]]


In [2]:
rng_state

('MT19937',
 array([2147483648, 3338624590, 3039918631, 3599810215, 1460170653,
         126009875, 1584965342, 4200159118, 1014255681,   30814631,
        1670608933, 3024697150, 3763829826, 2650304681,   18683483,
        1743359742,  297581065, 1912863282, 3356488665, 2356481858,
        1715381035, 2711368845, 3659778885, 3330146985, 2998034951,
        1046978966, 4126119321, 1153470135, 3445301710, 3004527812,
        2969745668, 4242450633, 2404209528,  832146728, 3511194078,
        4214324998,  884795177,  655566978, 2988861440, 4167364019,
         977634160,  141892612, 4152572597, 1873118418, 3253962982,
        3474708554,  755361219, 1065360023,    1896617, 3772239078,
        1327934064, 3587682528, 1893498057, 3087558425, 2649345807,
         432455441, 2686957458, 2644729426, 2936887035, 3372647793,
        4257899103, 4064905624, 1499789538,  232774600, 1033573550,
        3665431755, 3358491192, 3144637784,    9705473, 1230285509,
         741496724, 3767441241, 3670